## Describing final layer neurons of ResNet-18 (Places)

In [1]:
import os
#virtually move to parent directory
os.chdir("..")

import torch
import pandas as pd

from sentence_transformers import SentenceTransformer

import clip
import utils
import data_utils
import similarity

2023-03-22 07:19:55.947759: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
#Arguments
clip_name = 'ViT-B/16'
target_name = 'resnet18_places'
target_layer = 'fc'
d_probe = 'broden'
concept_set = 'data/broden_labels_clean.txt'
batch_size = 200
device = 'cuda'
pool_mode = 'avg'

save_dir = 'saved_activations'
similarity_fn = similarity.soft_wpmi

In [3]:
utils.save_activations(clip_name = clip_name, target_name = target_name, target_layers = [target_layer], 
                       d_probe = d_probe, concept_set = concept_set, batch_size = batch_size, 
                       device = device, pool_mode=pool_mode, save_dir = save_dir)

In [4]:
save_names = utils.get_save_names(clip_name = clip_name, target_name = target_name,
                                  target_layer = target_layer, d_probe = d_probe,
                                  concept_set = concept_set, pool_mode=pool_mode,
                                  save_dir = save_dir)

target_save_name, clip_save_name, text_save_name = save_names

similarities, target_feats = utils.get_similarity_from_activations(target_save_name, clip_save_name, 
                                                        text_save_name, similarity_fn, device=device)

with open(concept_set, 'r') as f: 
    words = (f.read()).split('\n')

100%|██████████| 365/365 [00:00<00:00, 1611.75it/s]

torch.Size([365, 1197])


In [5]:
#Clean up names of target classes
with open('data/categories_places365.txt', 'r') as f:
    cls_id_to_name = f.read().split('\n')
    cls_id_to_name = [(cls[3:]).split(' ')[0] for cls in cls_id_to_name]

def process_word(word):
    if concept_set.startswith('data/broden_labels'):
        if word.endswith("-s"):
            word = word[:-2]
        word = word.replace('_', ' ')
        return "{}".format(word)
    elif concept_set == 'data/categories_places365.txt':
        
        word = word[3:].split(' ')[0]
        word = word.replace('/', '-')
        word = word.replace('_', ' ')
        
        return "{}".format(word)

# Accuracies

In [6]:
id_to_label = data_utils.get_places_id_to_broden_label()

def clean_label(label):
    if label.startswith('/'):
        label = label[3:]
        label = label.split(' ')[0]
    if label.endswith('-s'):
        label = label[:-2]
    return label
    
def get_topk_acc(similarities, k=5):
    correct = 0
    total = 0
    for orig_id in range(len(similarities)):
        #skip classes not in Broden
        if id_to_label[orig_id]==None:
            continue
        else:
            vals, ids = torch.topk(similarities[orig_id], k, largest=True)
            for idx in ids[:k]:
                if (process_word(words[idx])==process_word(id_to_label[orig_id])):
                    correct += 1
                    continue
            total += 1
    return (correct/total)*100

In [7]:
print("CLIP-Dissect Top 1 acc:{:.4f}".format(get_topk_acc(similarities, k=1)))
print("CLIP-Dissect Top 5 acc:{:.4f}".format(get_topk_acc(similarities, k=5)))

CLIP-Dissect Top 1 acc:58.0524
CLIP-Dissect Top 5 acc:86.1423


In [8]:
df = pd.read_csv('data/NetDissect_results/resnet18_places365_fc.csv')
correct = 0
total = 0
for i, label in enumerate(df['label']):
    if id_to_label[i]==None:
        continue
    else:
        correct += (clean_label(label)==clean_label(id_to_label[i]))
        total += 1

print("Network Dissection Top 1 acc:{:.4f}".format(correct/total*100))

Network Dissection Top 1 acc:43.8202


# Cos similarities

In [9]:
model = SentenceTransformer('all-mpnet-base-v2')
clip_model, _ = clip.load(clip_name, device=device)

In [10]:
clip_preds = torch.argmax(similarities, dim=1)
clip_preds = [words[int(pred)] for pred in clip_preds]

clip_cos, mpnet_cos = utils.get_cos_similarity(clip_preds, cls_id_to_name, clip_model, model, device, batch_size)
print("CLIP-Dissect - Clip similarity: {:.4f}, mpnet similarity: {:.4f}".format(clip_cos, mpnet_cos))

CLIP-Dissect - Clip similarity: 0.9106, mpnet similarity: 0.7024


In [11]:
netdissect_res = pd.read_csv('data/NetDissect_results/resnet18_places365_fc.csv')
nd_preds = netdissect_res['label'].values
nd_preds = [clean_label(pred) for pred in nd_preds]

clip_cos, mpnet_cos = utils.get_cos_similarity(nd_preds, cls_id_to_name, clip_model, model, device, batch_size)
print("Network Dissection - Clip similarity: {:.4f}, mpnet similarity: {:.4f}".format(clip_cos, mpnet_cos))

Network Dissection - Clip similarity: 0.8887, mpnet similarity: 0.6697
